In [21]:
import os
import json
import requests
import pandas as pd

from serpapi import GoogleSearch

from dotenv import load_dotenv

from tqdm.notebook import tqdm
tqdm.pandas()

attempt 1

In [22]:
# Top Chess Players in the world as of December 16, 2024
# Ordering within the dictionary does not matter
top_players =[
    {'name': 'Magnus_Carlsen', 'fide_number': '1503014', 'chess_com_username': 'magnuscarlsen'},
    {'name': 'Fabiano_Caruana', 'fide_number': '2020009', 'chess_com_username': 'fabianocaruana'},
    {'name': 'Hikaru_Nakamura', 'fide_number': '2016192', 'chess_com_username': 'hikaru'},
    {'name': 'Arjun_Erigaisi', 'fide_number': '35009192', 'chess_com_username': 'ghandeevam2003'},
    {'name': 'Gukesh_Dommaraju', 'fide_number': '46616543', 'chess_com_username': 'gukeshdommaraju'},
    {'name': 'Nodirbek_Abdusattorov', 'fide_number': '14204118', 'chess_com_username': 'chesswarrior7197'},
    {'name': 'Alireza-Firouzja', 'fide_number': '12573981', 'chess_com_username': 'firouzja2003'},
    {'name': 'Ian_Nepomniachtchi', 'fide_number': '4168119', 'chess_com_username': 'lachesisq'},
    {'name': 'Yi_Wei', 'fide_number': '8603405', 'chess_com_username': 'wei-yi'},
    {'name': 'Viswanathan_Anand', 'fide_number': '5000017', 'chess_com_username': 'thevish'}
]

In [23]:
import subprocess
import json

def fetch_fide_data(fide_number):
    try:
        # Run the `fide-ratings-scraper` command with the correct syntax
        result = subprocess.run(
            ["fide-ratings-scraper", "get", "info", str(fide_number)],
            capture_output=True,
            text=True,
            check=True
        )
        # Parse the JSON output
        data = json.loads(result.stdout)
        return data
    except subprocess.CalledProcessError as e:
        print(f"Command error: {e}")
        print(f"STDOUT: {e.stdout}")
        print(f"STDERR: {e.stderr}")
        return None
    except json.JSONDecodeError:
        print("Failed to decode JSON. Check if the command's output is valid JSON.")
        return None

# Example usage
fide_number = 1503014
fide_data = fetch_fide_data(fide_number)
print(fide_data)


{'name': 'Carlsen, Magnus ', 'federation': 'Norway', 'birth_year': 1990, 'sex': 'Male', 'title': 'Grandmaster', 'standard_elo': '', 'rapid_elo': '', 'blitz_elo': '', 'world_rank_all_players': 1, 'world_rank_active_players': 1, 'national_rank_all_players': 1, 'national_rank_active_players': 1, 'continental_rank_all_players': 1, 'continental_rank_active_players': 1}


In [25]:
import pandas as pd
import subprocess
import json

# List of FIDE IDs for the top players (example FIDE IDs)
top_10_fide_numbers = [player['fide_number'] for player in top_players]

def fetch_fide_data_with_history(fide_number):
    """
    Fetches both player info and historical ELO data for a given FIDE ID.
    Combines the two datasets into a list of dictionaries for each historical record.
    """
    try:
        # Fetch player info
        info_result = subprocess.run(
            ["fide-ratings-scraper", "get", "info", str(fide_number)],
            capture_output=True,
            text=True,
            check=True
        )
        player_info = json.loads(info_result.stdout)

        # Fetch historical data
        history_result = subprocess.run(
            ["fide-ratings-scraper", "get", "history", str(fide_number)],
            capture_output=True,
            text=True,
            check=True
        )
        history_data = json.loads(history_result.stdout)

        # Check if history_data is a list or dictionary
        if isinstance(history_data, list):
            records = history_data  # If it's a list, use it directly
        elif isinstance(history_data, dict) and "history" in history_data:
            records = history_data["history"]  # If it's a dict, extract "history"
        else:
            print(f"Unexpected format for historical data: {history_data}")
            return []

        # Combine player info with history records
        combined_data = []
        for record in records:
            combined_data.append({
                'fide_id': fide_number,
                'name': player_info.get('name', ''),
                'federation': player_info.get('federation', ''),
                'world_rank_active_players': player_info.get('world_rank_active_players', ''),
                'date': record.get('date', ''),
                'standard': record.get('standard', None),
                'rapid': record.get('rapid', None),
                'blitz': record.get('blitz', None)
            })
        return combined_data

    except subprocess.CalledProcessError as e:
        print(f"Command error for FIDE ID {fide_number}: {e}")
        print(f"STDOUT: {e.stdout}")
        print(f"STDERR: {e.stderr}")
        return []

    except json.JSONDecodeError:
        print(f"Failed to decode JSON for FIDE ID {fide_number}. Check the command's output.")
        return []

# Fetch data for each FIDE ID
all_data = []
for fide_number in top_10_fide_numbers:
    player_data = fetch_fide_data_with_history(fide_number)
    all_data.extend(player_data)

# Convert combined data to a pandas DataFrame
df_combined = pd.DataFrame(all_data)

# Display the DataFrame
print(df_combined)


      fide_id                 name federation  world_rank_active_players  \
0     1503014     Carlsen, Magnus      Norway                          1   
1     1503014     Carlsen, Magnus      Norway                          1   
2     1503014     Carlsen, Magnus      Norway                          1   
3     1503014     Carlsen, Magnus      Norway                          1   
4     1503014     Carlsen, Magnus      Norway                          1   
...       ...                  ...        ...                        ...   
1779  5000017  Anand, Viswanathan       India                         10   
1780  5000017  Anand, Viswanathan       India                         10   
1781  5000017  Anand, Viswanathan       India                         10   
1782  5000017  Anand, Viswanathan       India                         10   
1783  5000017  Anand, Viswanathan       India                         10   

          date standard rapid blitz  
0     2025-Jan     2831  2819  2889  
1     2024-

In [26]:
df_combined

,fide_id,name,federation,world_rank_active_players,date,standard,rapid,blitz
0,1503014,"Carlsen, Magnus",Norway,1,2025-Jan,2831,2819,2889
1,1503014,"Carlsen, Magnus",Norway,1,2024-Dec,2831,2838,2890
2,1503014,"Carlsen, Magnus",Norway,1,2024-Nov,2831,2825,2893
3,1503014,"Carlsen, Magnus",Norway,1,2024-Oct,2831,2834,2888
4,1503014,"Carlsen, Magnus",Norway,1,2024-Sep,2832,2834,2888
...,...,...,...,...,...,...,...,...
1779,5000017,"Anand, Viswanathan",India,10,2001-Apr,2794,,
1780,5000017,"Anand, Viswanathan",India,10,2001-Jan,2790,,
1781,5000017,"Anand, Viswanathan",India,10,2000-Oct,2774,,
1782,5000017,"Anand, Viswanathan",India,10,2000-Jul,2762,,
